the plan: don't transform the features At All, one hot encode anything that can be one hot encoded, do a random forest over it

In [1]:
# xg boost?

In [1]:
import pandas as pd
import numpy as np
import sklearn # not actually needed

In [2]:
import pickle

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [4]:
og_df = pd.read_csv("../src/data/Traffic_Crashes_Resulting_in_Injury_20240518.csv")

/var/folders/j1/4b075fzj2xdbn0txbrmsq7hm0000gn/T/ipykernel_61742/2720964843.py:1: DtypeWarning: Columns (3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  og_df = pd.read_csv("../src/data/Traffic_Crashes_Resulting_in_Injury_20240518.csv")


In [7]:
og_df.columns

Index(['unique_id', 'cnn_intrsctn_fkey', 'cnn_sgmt_fkey', 'case_id_pkey',
       'tb_latitude', 'tb_longitude', 'geocode_source', 'geocode_location',
       'collision_datetime', 'collision_date', 'collision_time',
       'accident_year', 'month', 'day_of_week', 'time_cat', 'juris',
       'officer_id', 'reporting_district', 'beat_number', 'primary_rd',
       'secondary_rd', 'distance', 'direction', 'weather_1', 'weather_2',
       'collision_severity', 'type_of_collision', 'mviw', 'ped_action',
       'road_surface', 'road_cond_1', 'road_cond_2', 'lighting',
       'control_device', 'intersection', 'vz_pcf_code', 'vz_pcf_group',
       'vz_pcf_description', 'vz_pcf_link', 'number_killed', 'number_injured',
       'street_view', 'dph_col_grp', 'dph_col_grp_description',
       'party_at_fault', 'party1_type', 'party1_dir_of_travel',
       'party1_move_pre_acc', 'party2_type', 'party2_dir_of_travel',
       'party2_move_pre_acc', 'point', 'data_as_of', 'data_updated_at',
       'data_

In [31]:
# things to drop
df = og_df.drop(columns=['cnn_intrsctn_fkey', 'cnn_sgmt_fkey', 'case_id_pkey', 
                      'tb_latitude', 'tb_longitude', 'geocode_source', 
                      'officer_id', 'beat_number', 'direction', 
                      'control_device', 'vz_pcf_group', 'vz_pcf_description', 
                      'vz_pcf_link', 'number_killed', 'number_injured', 
                      'street_view', 'dph_col_grp', 
                      'party1_dir_of_travel', 'party1_move_pre_acc', 
                      'party2_dir_of_travel', 'party2_move_pre_acc', 
                      'point', 'data_as_of', 'data_updated_at', 
                      'data_loaded_at', 'Neighborhoods', 'SF Find Neighborhoods', 
                      'Current Police Districts', 'Current Supervisor Districts', 'Analysis Neighborhoods',
                      'reporting_district',
                     'geocode_location', 'juris', 
                         'collision_date', 'collision_time', 'accident_year', 'collision_datetime']) # drop these bc not easy to encode
df = df.reset_index() # this is so we can concatenate the one hot encoded stuff

# important note: we are removing collision time, so we don't have to remove the collision time NaNs. 
# would have to drop things later

df = df.drop(columns='index') # due to resetting index

one hot encode everything

In [33]:
# dropped collision_datetime because it won't convert to float nicely...
# df['collision_datetime'] = pd.to_datetime(df['collision_datetime'])

In [34]:
dont_encode = ['unique_id', 
                'distance', 
                'party_at_fault',
                'collision_severity']
cols_to_encode = list(df.columns)
for col in dont_encode:
    cols_to_encode.remove(col)
ct = ColumnTransformer([('onehoteverything', OneHotEncoder(), cols_to_encode)], sparse_threshold=0) # not sparse

In [35]:
df_encoded = pd.concat([df[dont_encode].reset_index().drop(columns='index'), pd.DataFrame(df_transformed)], axis = 1)


split things 

In [56]:
df_encoded = df_encoded.dropna(axis="index", subset=["party_at_fault", "distance"]) # whatever
df_train, df_test = train_test_split(df_encoded, test_size = 0.2, random_state = 42, shuffle = True) # i will never touch this
with open('df_test.pkl', 'wb') as handle:
    pickle.dump(df_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [57]:
X = df_train.drop(columns='collision_severity')
y = df_train['collision_severity']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42, shuffle = True)

fit model

In [58]:
from sklearn.ensemble import RandomForestClassifier

In [59]:
rfc = RandomForestClassifier()

In [60]:
rfc.fit(X_train, y_train)

/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


RandomForestClassifier()

In [63]:
y_pred = rfc.predict(X_val) # predicted

/opt/homebrew/lib/python3.10/site-packages/sklearn/utils/validation.py:1858: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [65]:
sum(y_pred == y_val)/len(y_val)

0.6430084745762712

In [68]:
np.unique(y_pred)

array(['Fatal', 'Injury (Complaint of Pain)', 'Injury (Other Visible)',
       'Injury (Severe)'], dtype=object)

In [69]:
np.unique(y_val)

array(['Fatal', 'Injury (Complaint of Pain)', 'Injury (Other Visible)',
       'Injury (Severe)'], dtype=object)

In [70]:
from sklearn.metrics import confusion_matrix

In [72]:
confusion_matrix(y_val, y_pred, labels = ['Fatal', 'Injury (Complaint of Pain)', 'Injury (Other Visible)',
       'Injury (Severe)'])

array([[   0,   54,   13,    0],
       [   1, 4956,  434,    2],
       [   0, 1939,  503,    3],
       [   0,  467,  120,    4]])